In [ ]:
import arcpy
import pandas as pd

# State Percentage Votes Difference

In [ ]:
import pandas as pd
import arcgis
import arcpy

In [ ]:
# Convert feature class to Spatially-enabled Dataframe
data_fc = r"path"
sedf = pd.DataFrame.spatial.from_featureclass(data_fc)
sedf.head()

In [ ]:
# Get list of states
states_list = sedf['state'].unique().tolist()
states_list

In [ ]:
# Set reference to each dataset that needs this calculated data
enriched_fc = data_fc
selected_fc = r"path"
tracts_fc = r"path"

In [ ]:
# For each state
for state in states_list:
    print("Calculating {0}...".format(state))

    # Get the total votes
    state_sedf = sedf.loc[sedf['state'] == state]
    state_votes_total = state_sedf['votes_total'].sum()
    
    # Get total votes for each party
    state_votes_dem_total = state_sedf['votes_dem'].sum()
    state_votes_gop_total = state_sedf['votes_gop'].sum()
    # Calculate total votes diff for state
    state_votes_total_diff = abs(state_votes_dem_total - state_votes_gop_total)
    
    # Calculate percent for each party
    state_votes_dem_pcnt = state_votes_dem_total / state_votes_total
    state_votes_gop_pcnt = state_votes_gop_total / state_votes_total
    # Calculate percent diff for state
    state_votes_pcnt_diff = abs(state_votes_dem_pcnt - state_votes_gop_pcnt)  
    
    # Write data for all county records for this state
    print("\tCalculating Enriched FC...")
    whereclause = "{0} = '{1}'".format(arcpy.AddFieldDelimiters(enriched_fc, "state"), state)
    with arcpy.da.UpdateCursor(enriched_fc, 
                               ['state_votes_total_diff', 'state_votes_pcnt_diff'],
                               whereclause) as cursor:
        for row in cursor:
            row[0] = state_votes_total_diff
            row[1] = state_votes_pcnt_diff
            cursor.updateRow(row)
            
    # Write data for all county records for this state
    print("\tCalculating Selected FC...")
    whereclause = "{0} = '{1}'".format(arcpy.AddFieldDelimiters(selected_fc, "state"), state)
    with arcpy.da.UpdateCursor(selected_fc, 
                               ['state_votes_total_diff', 'state_votes_pcnt_diff'],
                               whereclause) as cursor:
        for row in cursor:
            row[0] = state_votes_total_diff
            row[1] = state_votes_pcnt_diff
            cursor.updateRow(row)
            
    # Write data for all county records for this state
    print("\tCalculating Tracts FC...")
    whereclause = "{0} = '{1}'".format(arcpy.AddFieldDelimiters(tracts_fc, "State"), state)
    with arcpy.da.UpdateCursor(tracts_fc, 
                               ['state_votes_total_diff', 'state_votes_pcnt_diff'],
                               whereclause) as cursor:
        for row in cursor:
            row[0] = state_votes_total_diff
            row[1] = state_votes_pcnt_diff
            cursor.updateRow(row)